In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, col, lower, regexp_replace, split, to_date

spark= SparkSession.builder.config('spark.ui.port','4040').getOrCreate()

In [5]:
conf_series = spark.read.csv("file:///N/project/mag/mag-2021-01-05/mag/ConferenceSeries.txt",\
                         header = False,\
                         sep=r'\t')
#conf_series.printSchema()
conf_series.show(5)

+----------+-----+--------------+--------------------+----+----+-----+----------+
|       _c0|  _c1|           _c2|                 _c3| _c4| _c5|  _c6|       _c7|
+----------+-----+--------------+--------------------+----+----+-----+----------+
|1134804816|12796|         ICIDS|International Con...| 608| 607| 2683|2016-06-24|
|1165160117|14796|       SWAT4LS|Semantic Web Appl...|  81|  81|  197|2016-06-24|
|1192093291|12249|    TRIDENTCOM|Testbeds and Rese...| 570| 570| 5047|2016-06-24|
|1199066382|10257|BIOINFORMATICS|International Con...|9202|9202|14314|2016-06-24|
|1201746639|15533|           AIS|Autonomous and In...| 165| 165|  961|2016-06-24|
+----------+-----+--------------+--------------------+----+----+-----+----------+
only showing top 5 rows



In [3]:
def clean_text(c):
  c = regexp_replace(c, '"' , '')
  c = regexp_replace(c, '\\\\' , '')
  #c = regexp_replace(c, "(https?\://)\S+", "")
  #c = regexp_replace(c, "[^a-zA-Z0-9\\s]", "")
  #c = split(c, "\\s+") tokenization...
  return c

In [7]:
conf_series = conf_series.select(
                                 conf_series._c0.cast("string").alias("conferenceSeriesId"),
                                 conf_series._c1.cast("int").alias("rank"),
                                 conf_series._c2.cast("string").alias("normalizedName"),
                                 clean_text(col("_c3")).alias("displayName"),
                                 conf_series._c4.cast("int").alias("paperCount"),
                                 conf_series._c5.cast("int").alias("paperFamilyCount"),
                                 conf_series._c6.cast("int").alias("citationCount"),
                                 to_date(conf_series._c7, "yyyy-MM-dd").alias("createdDate")
)

conf_series.show(5)

+------------------+-----+--------------+--------------------+----------+----------------+-------------+-----------+
|conferenceSeriesId| rank|normalizedName|         displayName|paperCount|paperFamilyCount|citationCount|createdDate|
+------------------+-----+--------------+--------------------+----------+----------------+-------------+-----------+
|        1134804816|12796|         ICIDS|International Con...|       608|             607|         2683| 2016-06-24|
|        1165160117|14796|       SWAT4LS|Semantic Web Appl...|        81|              81|          197| 2016-06-24|
|        1192093291|12249|    TRIDENTCOM|Testbeds and Rese...|       570|             570|         5047| 2016-06-24|
|        1199066382|10257|BIOINFORMATICS|International Con...|      9202|            9202|        14314| 2016-06-24|
|        1201746639|15533|           AIS|Autonomous and In...|       165|             165|          961| 2016-06-24|
+------------------+-----+--------------+--------------------+--

In [8]:
%%time 
conf_series.coalesce(64).write.option("header","true") \
                               .option("sep","\t") \
                               .option("quoteAll", False) \
                               .option("emptyValue", None) \
                               .option("nullValue", None)\
                               .mode("overwrite") \
                               .csv('/N/project/mag/mag_jg_2021_update/nodes/conf_series')               

CPU times: user 4.75 ms, sys: 1.78 ms, total: 6.53 ms
Wall time: 1.44 s
